# Dataset Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

def analizza_dataset(path):
    """
    Analizza un dataset CSV e classifica ogni feature per tipo
    """
    # Carico il dataset
    df = pd.read_csv(path)
    
    print("="*60)
    print("ANALISI DATASET - CLASSIFICAZIONE FEATURE")
    print("="*60)
    
    print(f"\nDimensioni dataset: {df.shape[0]} righe, {df.shape[1]} colonne")
    
    # Dizionario per classificare le feature
    classificazione = {
        'BINARIE': [],
        'CATEGORICHE_BASSA_CARDINALITA': [],
        'CATEGORICHE_ALTA_CARDINALITA': [],
        'NUMERICHE_DISCRETE': [],
        'NUMERICHE_CONTINUE': [],
        'DATE/DATETIME': [],
        'TESTO_LUNGO': [],
        'ALTRO': []
    }
    
    # Analisi dettagliata per ogni colonna
    for col in df.columns:
        serie = df[col].dropna()  # Rimuovo valori nulli per l'analisi
        
        if len(serie) == 0:  # Colonna completamente vuota
            classificazione['ALTRO'].append((col, "VUOTA", 0, []))
            continue
            
        n_unique = serie.nunique()
        tipo_pandas = str(serie.dtype)
        esempi = list(serie.unique()[:5]) if n_unique <= 20 else list(serie.unique()[:3])           # Lista rappresentativa di valori della colonna
        
        # Classifico la feature
        if n_unique == 2:
            # Binaria
            classificazione['BINARIE'].append((col, tipo_pandas, n_unique, esempi))
            
        elif n_unique == 1:
            # Costante
            classificazione['ALTRO'].append((col, "COSTANTE", n_unique, esempi))
            
        elif 'datetime' in tipo_pandas.lower() or col.lower() in ['date', 'time', 'datetime']:
            # Data/Datetime
            classificazione['DATE/DATETIME'].append((col, tipo_pandas, n_unique, esempi))
            
        elif tipo_pandas == 'object':
            # Verifico se contiene stringhe lunghe (testo)
            lunghezza_media = serie.astype(str).str.len().mean() if len(serie) > 0 else 0
            
            if lunghezza_media > 50:  # Testo lungo
                classificazione['TESTO_LUNGO'].append((col, tipo_pandas, n_unique, esempi))
            elif n_unique <= 20:  # Soglia per categoriche a bassa cardinalità
                classificazione['CATEGORICHE_BASSA_CARDINALITA'].append((col, tipo_pandas, n_unique, esempi))
            else:
                classificazione['CATEGORICHE_ALTA_CARDINALITA'].append((col, tipo_pandas, n_unique, esempi))
                
        elif tipo_pandas in ['int64', 'int32', 'float64', 'float32']:
            # Numeriche
            if n_unique <= 20 and serie.dtype == 'int64':
                # Probabilmente discreta
                classificazione['NUMERICHE_DISCRETE'].append((col, tipo_pandas, n_unique, esempi))
            else:
                # Continua
                classificazione['NUMERICHE_CONTINUE'].append((col, tipo_pandas, n_unique, esempi))
        else:
            classificazione['ALTRO'].append((col, tipo_pandas, n_unique, esempi))
    
    # Stampo risultati
    for categoria, features in classificazione.items():
        if features:
            print(f"\n{categoria} ({len(features)} features):")
            print("-" * 50)
            for nome, tipo, n_val, esempi in features:
                print(f"  • {nome}")
                print(f"    Tipo: {tipo} | Valori unici: {n_val}")
                print(f"    Esempi: {esempi}")
                if nome in df.columns:
                    null_count = df[nome].isnull().sum()
                    null_perc = (null_count / len(df)) * 100
                    print(f"    Valori nulli: {null_count} ({null_perc:.1f}%)")
                print()
    
    # Sommario
    print("\n" + "="*60)
    print("SOMMARIO CLASSIFICAZIONE")
    print("="*60)
    totale = sum(len(features) for features in classificazione.values())
    for categoria, features in classificazione.items():
        if features:
            percentuale = (len(features) / totale) * 100
            print(f"{categoria:.<35} {len(features):>3} ({percentuale:5.1f}%)")
    
    return df, classificazione

def analisi_correlazioni_binarie(df, classificazione):
    """
    Analizza le correlazioni tra variabili binarie
    """
    binarie = [nome for nome, _, _, _ in classificazione['BINARIE']]
    
    if len(binarie) > 1:
        print(f"\n\nCORRELAZIONI TRA VARIABILI BINARIE ({len(binarie)} variabili):")
        print("-" * 60)
        
        df_bin = df[binarie]
        corr_matrix = df_bin.corr()
        
        # Trovo correlazioni forti (> 0.7 o < -0.7)
        correlazioni_forti = []
        for i in range(len(corr_matrix.columns)):
            for j in range(i+1, len(corr_matrix.columns)):
                corr = corr_matrix.iloc[i, j]
                if abs(corr) > 0.7:
                    correlazioni_forti.append((corr_matrix.columns[i], 
                                             corr_matrix.columns[j], corr))
        
        if correlazioni_forti:
            print("Correlazioni forti (|r| > 0.7):")
            for var1, var2, corr in correlazioni_forti:
                print(f"  {var1} <-> {var2}: {corr:.3f}")
        else:
            print("Nessuna correlazione forte trovata tra variabili binarie.")

def analisi_numeriche(df, classificazione):
    """
    Analisi delle variabili numeriche
    """
    numeriche = ([nome for nome, _, _, _ in classificazione['NUMERICHE_CONTINUE']] + 
                [nome for nome, _, _, _ in classificazione['NUMERICHE_DISCRETE']])
    
    if numeriche:
        print(f"\n\nSTATISTICHE VARIABILI NUMERICHE ({len(numeriche)} variabili):")
        print("-" * 60)
        
        stats = df[numeriche].describe()
        print(stats.round(2))

# Esecuzione dell'analisi
if __name__ == "__main__":
    
    file_path = "../updated_data___censored.csv"  
    
    try:
        df, classificazione = analizza_dataset(file_path)
        analisi_correlazioni_binarie(df, classificazione)
        analisi_numeriche(df, classificazione)
        
    except FileNotFoundError:
        print(f"File {file_path} non trovato. Assicurati che il path sia corretto.")
    except Exception as e:
        print(f"Errore durante l'analisi: {e}")

# FUNZIONI AGGIUNTIVE PER ANALISI SPECIFICA

def controlla_pattern_genetici(df):
    """
    Identifica pattern specifici per dati genetici (rs...)
    """
    colonne_genetiche = [col for col in df.columns if col.startswith('rs')]
    if colonne_genetiche:
        print(f"\n\nVARIABILI GENETICHE IDENTIFICATE ({len(colonne_genetiche)}):")
        print("-" * 50)
        for col in colonne_genetiche[:10]:  # Mostra primi 10
            valori = df[col].value_counts().head()
            print(f"{col}: {dict(valori)}")

def controlla_variabili_tempo(df):
    """
    Identifica variabili legate al tempo
    """
    colonne_tempo = [col for col in df.columns if any(keyword in col.lower() 
                    for keyword in ['time', 'date', 'month', 'year', 'age', 'duration'])]
    
    if colonne_tempo:
        print(f"\n\nVARIABILI TEMPORALI IDENTIFICATE ({len(colonne_tempo)}):")
        print("-" * 50)
        for col in colonne_tempo[:10]:
            if col in df.columns:
                print(f"{col}: range {df[col].min()} - {df[col].max()}")

# Eseguo anche queste analisi specifiche
if __name__ == "__main__":
    try:
        controlla_pattern_genetici(df)
        controlla_variabili_tempo(df)
    except:
        pass

/tmp/ipykernel_4036518/2039534506.py:12: DtypeWarning: Columns (11,107,109,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


ANALISI DATASET - CLASSIFICAZIONE FEATURE

Dimensioni dataset: 502412 righe, 306 colonne

BINARIE (137 features):
--------------------------------------------------
  • Sex
    Tipo: int64 | Valori unici: 2
    Esempi: [1, 0]
    Valori nulli: 0 (0.0%)

  • PW_abnp_0
    Tipo: float64 | Valori unici: 2
    Esempi: [0.0, 1.0]
    Valori nulli: 331698 (66.0%)

  • Above_moderate_vigorous
    Tipo: float64 | Valori unici: 2
    Esempi: [1.0, 0.0]
    Valori nulli: 100106 (19.9%)

  • Above_walking_moderate_vigorous
    Tipo: float64 | Valori unici: 2
    Esempi: [1.0, 0.0]
    Valori nulli: 100218 (19.9%)

  • Quality_IMT_120
    Tipo: float64 | Valori unici: 2
    Esempi: [1.0, 2.0]
    Valori nulli: 499834 (99.5%)

  • Quality_IMT_150
    Tipo: float64 | Valori unici: 2
    Esempi: [1.0, 2.0]
    Valori nulli: 499834 (99.5%)

  • Quality_IMT_210
    Tipo: float64 | Valori unici: 2
    Esempi: [1.0, 2.0]
    Valori nulli: 499834 (99.5%)

  • Quality_IMT_240
    Tipo: float64 | Valori uni

In [2]:
# Top 20 per % missing
missing = (df.isnull().mean()*100).sort_values(ascending=False).head(50)
print("\nTOP 20 COLONNE PER MISSING (%):")
print(missing.to_string())


TOP 20 COLONNE PER MISSING (%):
Creat_urine_flag_0                                          99.999204
Months_from_first_malnutrition_DM                           99.998607
Wbody_impedance_manual_0                                    99.995024
Arm_left_impendance_manual_0                                99.995024
Leg_right_impendance_manual_0                               99.995024
Leg_left_impendance_manual_0                                99.995024
Arm_right_impendance_manual_0                               99.995024
Med_cbde_3_0                                                99.981489
Months_from_first_hypertension_pregnancy                    99.974125
PWV_manual_0                                                99.967955
Months_from_first_secondary_hypertension                    99.940686
Months_from_first_other_DM                                  99.891523
Months_from_first_yes_proteinuria_hypertension_pregnancy    99.885154
Months_from_first_DM_pregnancy                           